### code from https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/rag-intro.ipynb

In [1]:
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py

--2024-09-19 13:47:34--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3832 (3.7K) [text/plain]
Saving to: ‘minsearch.py.2’

minsearch.py.2      100%[===================>]   3.74K  --.-KB/s    in 0.001s  

2024-09-19 13:47:34 (5.43 MB/s) - ‘minsearch.py.2’ saved [3832/3832]



In [2]:
import minsearch

In [3]:
import json

In [4]:
with open('documents.json','rt') as f_in:
    docs_raw = json.load(f_in)

In [5]:
documents =[]
for docs in docs_raw:
    for doc in  docs["documents"]: 
        doc["course"] =docs["course"]
        documents.append(doc)

In [6]:
documents[0].keys()

dict_keys(['text', 'section', 'question', 'course'])

In [7]:
from minsearch import Index

index = Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

In [8]:
index.fit(documents)

In [37]:
query = "The course has already started can i still enroll?"

filter_dict = {"course": "data-engineering-zoomcamp"}
boost_dict = {"question": 3, "section": 0.5}

results = index.search(
    query,
    filter_dict, 
    boost_dict
)

for result in results:
    print(result)

{'text': "Yes, even if you don't register, you're still eligible to submit.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.", 'section': 'General course-related questions', 'question': 'Course - Can I still join the course after the start date?', 'course': 'data-engineering-zoomcamp'}
{'text': 'Yes, we will keep all the materials after the course /finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.', 'section': 'General course-related questions', 'question': 'Course - Can I follow the course after it finishes?', 'course': 'data-engineering-zoomcamp'}
{'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nGit\nLook

In [49]:
prompt_template="""
You are a teaching assistant, Please answer the QUESTION based on facts from the CONTEXT, 
If CONTEXT doesnot have the facts, Please answer with NONE.

QUESTION: {question}

CONTEXT: {context}
""".strip()

In [50]:
from ai21 import AI21Client
from ai21.models.chat import UserMessage

# One way of passing your key to the client.
import os
AI21_API_KEY = os.environ["AI21_API_KEY"]
client = AI21Client(api_key=AI21_API_KEY)

def single_message_instruct(content):
    messages = [
        UserMessage(
            content=content
        )
    ]

    response = client.chat.completions.create(
        model="jamba-1.5-large",
        messages=messages,
        top_p=1.0 # Setting to 1 encourages different responses each call.
    )
    return response.to_json()


In [40]:
response = single_message_instruct(query)

/tmp/ipykernel_7670/168767803.py:21: DeprecationWarning: The 'to_json' method is deprecated and will be removed in a future version. Please use Pydantic's built-in methods instead.
  return response.to_json()


In [41]:
import json
json_response = json.loads(response)
json_response["choices"][0]["message"]["content"]

'Yes, you can still enroll in the course. Please provide me with your email address so I can send you the enrollment link.'

In [42]:
def something_new(content):
    response = single_message_instruct(content)
    json_response = json.loads(response)
    content = json_response["choices"][0]["message"]["content"]
    return content

In [43]:
print(something_new(query))

It depends on the course and the institution offering it. Many courses allow late enrollment, especially if they are online or have rolling admissions. Here are a few steps you can take to find out:

1. **Check the Course Website**: Look for information about enrollment deadlines and policies.
2. **Contact the Instructor or Administrator**: Send an email or call to ask if late enrollment is possible.
3. **Visit the Institution**: If it's a local institution, visit the admissions or registrar's office to inquire about enrolling late.
4. **Look for Similar Courses**: If this course doesn't allow late enrollment, there might be similar courses available that do.

Would you like help finding contact information or drafting an email to inquire about enrollment?


/tmp/ipykernel_7670/168767803.py:21: DeprecationWarning: The 'to_json' method is deprecated and will be removed in a future version. Please use Pydantic's built-in methods instead.
  return response.to_json()


In [44]:
context =""
for doc in results:
    context =context+f"section: {doc['section']} \nquestion: {doc["question"]}\n answer: {doc["text"]} \n\n"
    

In [57]:
prompt = prompt_template.format(question=query , context=context).strip()

In [58]:
print(prompt)

You are a teaching assistant, Please answer the QUESTION based on facts from the CONTEXT, 
If CONTEXT doesnot have the facts, Please answer with NONE.

QUESTION: The course has already started can i still enroll?

CONTEXT: section: General course-related questions 
question: Course - Can I still join the course after the start date?
 answer: Yes, even if you don't register, you're still eligible to submit.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute. 

section: General course-related questions 
question: Course - Can I follow the course after it finishes?
 answer: Yes, we will keep all the materials after the course /finishes, so you can follow the course at your own pace after it finishes.
You can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project. 

section: General course-related questions 
question: Co

In [60]:
print(something_new(prompt))

Yes, even if you don't register, you're still eligible to submit. Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.


/tmp/ipykernel_7670/168767803.py:21: DeprecationWarning: The 'to_json' method is deprecated and will be removed in a future version. Please use Pydantic's built-in methods instead.
  return response.to_json()
